**I work on the base environment**

In [1]:
import pandas as pd
import os

In [16]:
#Create an empty dataframe to add the predictions and the country code - till when we have the ground truth data
df_preds_all = pd.DataFrame(columns=['country_code','gpi_predicted'])
l_countrypreds = [] #list to add the countries for which we have predictions
count = 0
for i in os.listdir('XGBoost_results/'):
    if i.endswith('_xgb_predictions.csv'):
        country = i.split('_')[0]
        
        if country !='WE' and country != 'SY':
            
            preds = pd.read_csv('XGBoost_results/'+ i , index_col = 0)
            
            ##add the new predictions
            #newpreds = pd.read_csv('../../GPI_project/new_2021/XGBoost_results_2021/%s_xgb_predictions.csv' %country, index_col = 0)
            
            #preds = preds.append(newpreds)
            
            #Get the length of the predictions. It is enough to read one prediction file since all predictions' length is the same.
            if count == 0:
                length = len(preds)
                count = count+1
            df_preds = pd.DataFrame({'gpi_predicted':preds.prediction1}).reset_index(drop = True)
            l_country_name = [country] * length 
            df_country = pd.DataFrame({'country_code':l_country_name})

            #Merge the predictions dataframe with the corresponding country code in the form of dataframe
            country_preds = pd.merge(df_preds, df_country, left_index=True, right_index=True)
            df_preds_all = df_preds_all.append(country_preds, sort=False)

            l_countrypreds.append(country)
            

Reset the index on the dataframe with all the predictions per country:

In [18]:
df_preds_all.reset_index(drop = True, inplace = True)

In [19]:
#Create an empty dataframe to add the real GPI score, the monthyear and the country code
df_interp_all = pd.DataFrame(columns=['country_code','MonthYear','GPI_score'])
l_country_interp = [] #list to add the countries for which we have real gpi

for i in os.listdir('data/interpolated_gpi/'):
    if i.startswith("interpolated"):
        
        #Get the country code
        country = i.split('_')[-1].split('.')[0]
        
        if country !='WE' and country != 'SY' :
            #Create a list with the country name to later append on the main dataframe
            l_country_name = [country] * length
            df_country = pd.DataFrame({'country_code':l_country_name})

            #Read the file 
            file = pd.read_csv('data/interpolated_gpi/' + i , index_col = 0)
            #Get the test set length of the file that correspond to the length of the predicted gpi
            t_file = file.tail(length)
            #Reset the index of the t_file
            t_file = t_file.reset_index()

            #Merge the real gpi with the country code in the form of a dataframe
            country_interp = pd.merge(t_file, df_country, left_index=True, right_index=True)
            df_interp_all = df_interp_all.append(country_interp, sort=False)

            l_country_interp.append(country)

Reset the index on the dataframe with all the predictions per country:

In [20]:
df_interp_all.reset_index(drop = True, inplace = True)

In [21]:
len(df_interp_all), len(df_preds_all)

(13685, 13685)

Now group the two dataframes based on the country code:

In [22]:
grouped1 = df_interp_all.groupby('country_code') 

In [23]:
grouped2 = df_preds_all.groupby('country_code') 

Read the fips with the countries dataframe that will be used below:

In [25]:
df_fips = pd.read_csv('data/Fips10_4.csv')

In [26]:
df_fips.head(3)

,Country,FIPS_10_4
0,Afghanistan,AF
1,Aland Islands,–
2,Albania,AL


Get the extra the forecast dates that will be used for the final dataframe:

In [27]:
all_var = pd.read_csv('data/all_variables_and_GPI_monthly_all_countries/all_variables_%s.csv'%country, index_col=0)
all_var.index.to_list()

f_date = 202104 #set the first forecast date
l_date = 202110 #set last+1 forecast date (to get the last forecast)
df = all_var.index.to_list().index(f_date)
dl = all_var.index.to_list().index(l_date)
f_dates = all_var.index.to_list()[df:dl]
n_f = 6 #set the number of forecasts ahead

In [28]:
f_dates

[202104, 202105, 202106, 202107, 202108, 202109]

In [33]:
#Create an empty dataframe that will be the final one to add the forecasts as well
final = pd.DataFrame(columns=['country_code', 'MonthYear', 'GPI_score', 'gpi_predicted'])

#For each country merge the dataframes above
for country in l_country_interp: 
    df1 = grouped1.get_group(country).reset_index(drop=True)
    df2 = grouped2.get_group(country).reset_index(drop=True).drop(['country_code'], axis=1)
    df_both = pd.merge(df1, df2, left_index=True, right_index=True)
    
    #Prepare the dataframe with new forecast values
    forecasts = pd.read_csv('XGBoost_forecasts/%s_xgb_predictions.csv' %country, index_col = 0)
    forecasts = forecasts.iloc[:,:n_f+1]
    
    forecasts = forecasts.iloc[:,1:].T.reset_index(drop = True)
    flength = len(forecasts)
    forecasts.columns = ['gpi_predicted']
    forecasts['MonthYear'] = f_dates
    forecasts['country_code'] = flength * [country]  
    
    #Append the forecasts to the dataframe
    df_both = df_both.append(forecasts)
    df_both.reset_index(drop = True, inplace = True)
    
    #Create an extra column with the country corresponding to the fips code
    country_fips = df_fips.at[df_fips['FIPS_10_4'].eq(country).idxmax(),'Country']
    fips_country_name = [country_fips] * len(df_both)
    df_country = pd.DataFrame({'country_name':fips_country_name})
    
    #Merge df_both with df_country
    df_both_country = pd.merge(df_country, df_both, left_index=True, right_index=True)
    
    #Append on the final dataframe
    final = final.append(df_both_country, sort=False)
    

We have been asked to add to the dataframe the variables importance. The following produces this information to add them to the dataframe.

In [35]:
l_train_date = 202103 #set last training date, which corresponds to the last date of the ground truth data

In [57]:
#Create an empty dataframe to add all important variables for all countries
var_all = pd.DataFrame()
#Create a dataframe with all important variables and the country code
for i in os.listdir('XGBoost_results/'):
    
    if i.endswith('_xgb_impvar.csv'):
        country = i.split('_')[0]
        
        if country !='WE' and country != 'SY':
            var = pd.read_csv('XGBoost_results/'+ i , index_col = 0)
            
            #add the variable importance used for the forecast
            newvar = pd.read_csv('XGBoost_forecasts/%s_xgb_impvar.csv' %country, index_col = 0)
            newvar['MonthYear'] = [l_train_date] * len(newvar)
                        
            #Append the forecasted values as well
            var = var.append(newvar)          
            #Use pivot so that you have as column names the variables and rows their importance/value
            var = var.pivot(index='MonthYear', columns='Variable', values='Importance')
            #Add the country code to the dataframe
            var['country_code'] = [country] * len(var)
            
            #Now, delete the MonthYear column. You can anyway merge with the final dataframe, since the importance is ordered by date
            var.reset_index(inplace = True)
            
            #Get as MonthYear the date of the prediction and not the date of the last training
            #Get any column of 'MonthYear' for the matching since it is the same
            var['MonthYear'] = country_interp['MonthYear'].append(pd.Series(f_dates)).reset_index(drop=True)
            
            #Create the dataframe with the same forecast variables 
            var2 = pd.concat([pd.DataFrame(var.iloc[-1]).T.iloc[:,1:]] * (len(f_dates[1:])))
            var2.reset_index(drop = True, inplace = True)
            var2['MonthYear'] = f_dates[1:]
            
            #Append to the biggere dataframe
            var = var.append(var2)
            
            #Merge the country dataframe of variables importance with the dataframe that contains all country var imp
            var_all = var_all.append(var, sort=False)
            

In [95]:
len(final), len(var_all)

(14651, 14651)

Now, merge the variable importance information with the final dataframe.

In [60]:
final = pd.merge(final, var_all, on=['country_code','MonthYear'])

In [62]:
len(final)

14651

Reorganise the columns:

In [63]:
columns = final.columns.to_list()

In [64]:
columns = columns[1:2] + columns[4:5] + columns[0:1] + columns[2:4] + columns[5:]

In [65]:
final = final[columns]

In [66]:
len(final.country_code.unique())

161

Round the numbers:

In [67]:
final.iloc[:,3:] = round(final.iloc[:,3:],4)

Remove the prefix from the columns:

In [68]:
col_new = final.iloc[:,5:].columns.str.lstrip('event_count_').to_list()

Make the columns with numbers from strings to integers:

In [69]:
col_new = [int(x) for x in col_new]

Create the new column names to replace the old:

In [70]:
col_new2 = final.columns.to_list()[:5] + col_new 

Replace the old column names:

In [71]:
final.columns = col_new2

Reorder the columns with the event numbers in ascending order so that we can merge and match easily the descriptive titles:

In [72]:
col_new3 = final.columns[:5].to_list() + final.columns[5:].sort_values().to_list()

In [73]:
final = final[col_new3]


Now add the descriptive names to the variables:

In [77]:
name_variables = pd.read_csv('description_variables.csv')

Get the column names that should be made descriptive:

In [78]:
columns_change = final.columns[5:].to_list()

Sort the columns so that you can easily merge to match the descriptive names:

In [79]:
columns_change.sort()

In [80]:
columns_change = pd.DataFrame(columns_change, columns= ['CAMEOEVENTCODE'])

In [81]:
name_variables = name_variables.sort_values('CAMEOEVENTCODE').reset_index(drop=True)

In [82]:
columns_changed = pd.merge(columns_change, name_variables, on="CAMEOEVENTCODE")['EVENTDESCRIPTION']

In [83]:
columns_changed = columns_changed.to_list()

In [84]:
final_cols = final.columns.to_list()[:5] + columns_changed

In [85]:
final.columns = final_cols

In [86]:
final

,MonthYear,country_name,country_code,GPI_score,gpi_predicted,Make statement,Decline comment,Make pessimistic comment,Make optimistic comment,Consider policy option,...,Use conventional military force,"Impose blockade, restrict movement",Occupy territory,Fight with small arms and light weapons,Fight with artillery and tanks,Employ aerial weapons,Violate ceasefire,Engage in mass expulsion,Engage in mass killings,Engage in ethnic cleansing
0,201403,Spain,SP,1.6300,1.6914,0.00922191,0.00205365,0.00258881,0.00304693,0.00597635,...,0.00701164,0,0.00647943,0.00402723,0.0125938,0.00544799,NaN,NaN,NaN,NaN
1,201404,Spain,SP,1.6238,1.6610,0.0108516,0.00369934,0.0070494,0.00308904,0.0059206,...,0.00953077,0,0.000763006,0.00342654,0.0168576,0.0129909,NaN,NaN,NaN,NaN
2,201405,Spain,SP,1.6177,1.6513,0.000275281,0.00107858,0.000210615,0.00271787,0.000638663,...,0.00360465,0.00718811,0.000198101,0.0238128,0.0206324,0.00175264,NaN,NaN,NaN,NaN
3,201406,Spain,SP,1.6115,1.6550,0.000116735,0.00171243,0.00287457,0.000931874,0.000841625,...,0.0178504,0,2.48733e-05,0.0505367,0.0144891,0.00452208,NaN,NaN,NaN,NaN
4,201407,Spain,SP,1.6053,1.6552,0.00012881,0.000292653,0.000302728,0.00367897,0.00123307,...,0.0179736,0.000353525,0.00911597,0.0426761,0.0127407,0.00720818,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14646,202105,Indonesia,ID,NaN,1.7861,0,0.0147958,0.00190761,0.00437741,0.0419324,...,0.00610109,0.0196468,0.00466104,0,0,0.000225095,NaN,NaN,0,NaN
14647,202106,Indonesia,ID,NaN,1.7847,0,0.0147958,0.00190761,0.00437741,0.0419324,...,0.00610109,0.0196468,0.00466104,0,0,0.000225095,NaN,NaN,0,NaN
14648,202107,Indonesia,ID,NaN,1.7997,0,0.0147958,0.00190761,0.00437741,0.0419324,...,0.00610109,0.0196468,0.00466104,0,0,0.000225095,NaN,NaN,0,NaN
14649,202108,Indonesia,ID,NaN,1.8056,0,0.0147958,0.00190761,0.00437741,0.0419324,...,0.00610109,0.0196468,0.00466104,0,0,0.000225095,NaN,NaN,0,NaN


In [122]:
final.to_csv('data/all_countries_for_dashboard.csv', index=False)

In [3]:
a = pd.read_csv('data/all_countries_for_dashboard.csv', index_col=0)

In [7]:
a = a.loc[a['country_name'] == 'Ukraine']


In [20]:
perc_error = ((a.gpi_predicted - a.GPI_score)/a.GPI_score)*100

In [22]:
perc_error.min(), perc_error.max()

(-11.828929517410687, 6.918099463856538)

In [24]:
np.mean(perc_error)

-1.6301784998731965